<a href="https://colab.research.google.com/github/rshekhar0/mscb1/blob/main/Develop_a_transaction_class_to_send_and_receive_money_and_test_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import Crypto  # Importing the Crypto module for cryptographic functions
import binascii  # Importing the binascii module for binary-to-ASCII conversion
import datetime  # Importing the datetime module for handling date and time
import collections  # Importing the collections module for OrderedDict

from Crypto.PublicKey import RSA  # Importing the RSA class from the Crypto.PublicKey module for RSA key generation
from Crypto.Signature import PKCS1_v1_5  # Importing the PKCS1_v1_5 class from the Crypto.Signature module for RSA signature
from Crypto.Hash import SHA  # Importing the SHA hashing algorithm from the Crypto.Hash module

class Client:
    def __init__(self):
        # Creating a random number for key generation
        random = Crypto.Random.new().read
        # Generating a new public key and private key
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        # Converting the public key to DER format, then converting it to ASCII hex representation
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

class Transaction:
    def __init__(self, sender, receiver, value):
        self.sender = sender  # Setting the sender of the transaction
        self.receiver = receiver  # Setting the receiver of the transaction
        self.value = value  # Setting the value of the transaction
        self.time = datetime.datetime.now()  # Getting the current date and time

    def to_dict(self):
        # Checking if the sender is "Genesis"; if so, setting identity as "Genesis", else getting sender's identity
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        # Creating an ordered dictionary with transaction details
        return collections.OrderedDict({
            'sender': identity,
            'receiver': self.receiver,
            'value': self.value,
            'time': self.time
        })

    def sign_transaction(self):
        private_key = self.sender._private_key  # Getting the sender's private key
        signer = PKCS1_v1_5.new(private_key)  # Creating a signer object with the sender's private key
        h = SHA.new(str(self.to_dict()).encode('utf8'))  # Hashing the transaction details
        return binascii.hexlify(signer.sign(h)).decode('ascii')  # Signing the hash and returning the signature

Rahul = Client()  # Creating an instance of the Client class
print("-"*50)
print("Rahul Key")
print(Rahul.identity)  # Printing Rahul's public key

Golu = Client()  # Creating another instance of the Client class
print("-"*50)
print("Golu Key")
print(Golu.identity)  # Printing Golu's public key

t = Transaction(Rahul, Golu.identity, 10.0)  # Creating a transaction instance
print("-"*50)
print("Transaction Sign")
signature = t.sign_transaction()  # Generating the transaction signature
print(signature)  # Printing the transaction signature
print("-"*50)


--------------------------------------------------
Rahul Key
30819f300d06092a864886f70d010101050003818d0030818902818100cd76cbcebb2b65fba26f0803fb5bea112445c16fa166e95b9f9d2ddac9801b2fdd94f14179f1a9e41a87d70ccd1ecd73bdfdded28217f1b309ed63b97084a757bf5350b31670e052449c40b72ea7764357fd1dd5f348f3d33c791ee4fe924a948690fbf43445feedc2c52e0a6648287cd9bea26762f9df3c98f9e6666e5d676f0203010001
--------------------------------------------------
Golu Key
30819f300d06092a864886f70d010101050003818d0030818902818100a6c39637dc8653833cc664be60d3391e4ae34b1af9456320a4a3446d42a892789b255d47c6949aace829f5917d8a498522be8fe2db57ab1b7568dc07be51c78008282f1ae652ade36237bf678516557fb08600763e04373bf94108903f8958a45de49f31f96c419ed9d45bea1780c3c8a61e7a328512f86e34d4a8320046aa250203010001
--------------------------------------------------
Transaction Sign
59c4db93fe64967e3b11db6f8226821a1b17a25c3134f60261ce97aa1f5686620ceed9705d69bf7c0e41ce09c082f3c0bc556ff60c7e446e3bffd7e5e895bc3e00d4b9a46bb090aebe05d459a3e1933fd